In [ ]:
import gtdynamics as gtd
from gtdynamics import ContactGoal, PointOnLink, Slice, KinematicsSlice
from gtsam import Pose3, Point3

In [ ]:
# Load the vision 60 quadruped by Ghost robotics: https://youtu.be/wrBNJKZKg10
robot = gtd.CreateRobotFromFile(gtd.URDF_PATH + "/vision60.urdf");

In [ ]:
# establish contact/goal pairs
contact_in_com =(0.14, 0, 0)
contact_goals = [
    ContactGoal(PointOnLink(robot.link("lower1"), contact_in_com), [-0.4, 0.16, 0]),
    ContactGoal(PointOnLink(robot.link("lower0"), contact_in_com), [0.3, 0.16, 0]),
    ContactGoal(PointOnLink(robot.link("lower2"), contact_in_com), [0.3, -0.16, 0]),
    ContactGoal(PointOnLink(robot.link("lower3"), contact_in_com), [-0.4, -0.16, 0])
    ]
print(contact_goals)

In [ ]:
# Inverse kinematics for a single time slice
slice = Slice(777)
kinematics = KinematicsSlice(robot)
result = kinematics.inverse(slice, contact_goals)
print(result)

In [ ]:
for goal in contact_goals:
    print(goal.link().name(), goal.predict(result,k=777))

In [ ]:
# Inverse kinematics for a Phase
phase = gtd.Phase(7)
phase_kinematics = gtd.KinematicsPhase(robot)
    print(goal.link().name(), goal.satisfied(result,k=0,tol=1e-3))

In [ ]:
for goal in contact_goals:
    print(goal.link().name(), goal.predict(result,k=5))